In [1]:
import numpy as np
import scanpy as sc
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.sparse as sp
import os
from torch.utils.data import DataLoader
from torch.distributions import NegativeBinomial
from pyro.distributions.zero_inflated import ZeroInflatedNegativeBinomial

#calculate ARI score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, accuracy_score

In [2]:
torch.manual_seed(42)

## Dataset

In [3]:
import math
import pandas as pd
import numpy as np

tmp = []
tmplist = []
pd_celldata = pd.read_csv("proteinOutFileCat_nn.csv", sep=',', header=0)
pd_celldata = pd_celldata.rename({'Cancer Type': 'cancer_type'}, axis=1)

celldata = pd_celldata.to_numpy()

features = list(pd_celldata)

#get column names
i_d = []
for row in celldata:
    i_d.append(row[0])
    
# Remove first column: patients
i_d = i_d[1:]

celldata_t = celldata[:,:]
print(celldata_t) 

pd_celldata

[['TCGA-A2-A0D0-01' 0.7618 0.7481 ... 0.0132 1.6853 1]
 ['TCGA-BH-A0HK-01' 1.4195 0.9259 ... 0.0 0.9144 3]
 ['TCGA-C8-A12T-01' 0.318 0.0 ... 0.0 0.0087 2]
 ...
 ['TCGA-E2-A159-01' 0.7527 0.5561 ... 0.0 0.0 1]
 ['TCGA-A2-A0T3-01' 0.0126 0.0 ... 0.0628 0.4018 4]
 ['TCGA-A2-A0YD-01' 0.9947 0.407 ... 0.0 0.267 3]]


,Composite.Element.REF,A1BG,A2M,A2ML1,AAAS,AACS,AAED1,AAGAB,AAK1,AAMDC,...,ZSWIM8,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,cancer_type
0,TCGA-A2-A0D0-01,0.7618,0.7481,0.0000,1.3759,0.0,0.0000,0.0000,0.0160,0.0000,...,0.4656,0.2251,0.7310,0.0000,0.0000,1.1508,0.0000,0.0132,1.6853,1
1,TCGA-BH-A0HK-01,1.4195,0.9259,0.0000,0.6716,0.0,0.0000,0.0808,0.1219,0.0000,...,0.2386,0.0193,0.0000,0.0000,0.0000,0.6027,0.0000,0.0000,0.9144,3
2,TCGA-C8-A12T-01,0.3180,0.0000,0.0000,0.1638,0.0,0.0000,0.4738,0.0433,0.0000,...,0.0000,0.0000,0.2109,0.2311,0.0000,0.2617,0.3484,0.0000,0.0087,2
3,TCGA-A2-A0D2-01,0.0128,0.0000,0.0000,0.3808,0.0,0.3719,0.0000,0.0169,0.0000,...,0.0927,0.2363,0.7675,0.6338,0.0000,0.1919,0.3602,0.0000,0.3334,1
4,TCGA-C8-A12U-01,0.0000,0.1080,0.0000,0.0000,0.0,0.0000,0.1073,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.1598,0.5379,0.0397,0.0925,0.0000,0.1183,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,TCGA-BH-A18Q-01,0.0771,0.0000,0.7431,0.2510,0.0,0.0000,0.0000,0.1511,0.0000,...,0.1084,0.0598,0.0921,0.0000,0.1626,0.0000,0.0000,0.0586,0.2732,1
101,TCGA-C8-A130-01,0.0997,0.0000,0.0000,0.0000,0.0,0.0296,0.1863,0.1723,0.0922,...,0.1842,0.0000,0.0000,0.1901,0.1515,0.2357,0.0000,0.0000,0.0072,4
102,TCGA-E2-A159-01,0.7527,0.5561,3.3277,0.8935,0.0,0.0000,0.0000,0.0975,0.0000,...,0.4146,0.0000,0.9516,0.6781,0.0000,1.3542,0.0000,0.0000,0.0000,1
103,TCGA-A2-A0T3-01,0.0126,0.0000,0.0000,0.0796,0.0,0.0000,0.0000,0.2538,0.0000,...,0.0000,0.0000,0.3311,0.0874,0.0000,0.0000,0.0000,0.0628,0.4018,4


In [4]:
numlist=[]; obj_ind=[]
labels = []
for ind in range(celldata_t.shape[0]):
    x = celldata_t[ind,:].astype(np.str)
    numlist.append(x.astype(np.str))

i = 0
newList = []
for row in numlist:
    newList.append(row)
    labels.append(row[-1])
    i = i+1
    
    

numlist = newList
numlist = np.array(numlist)

pd_celldata_t = pd_celldata.drop(columns="Composite.Element.REF")


In [5]:
print(pd_celldata_t)

       A1BG     A2M   A2ML1    AAAS  AACS   AAED1   AAGAB    AAK1   AAMDC  \
0    0.7618  0.7481  0.0000  1.3759   0.0  0.0000  0.0000  0.0160  0.0000   
1    1.4195  0.9259  0.0000  0.6716   0.0  0.0000  0.0808  0.1219  0.0000   
2    0.3180  0.0000  0.0000  0.1638   0.0  0.0000  0.4738  0.0433  0.0000   
3    0.0128  0.0000  0.0000  0.3808   0.0  0.3719  0.0000  0.0169  0.0000   
4    0.0000  0.1080  0.0000  0.0000   0.0  0.0000  0.1073  0.0000  0.0000   
..      ...     ...     ...     ...   ...     ...     ...     ...     ...   
100  0.0771  0.0000  0.7431  0.2510   0.0  0.0000  0.0000  0.1511  0.0000   
101  0.0997  0.0000  0.0000  0.0000   0.0  0.0296  0.1863  0.1723  0.0922   
102  0.7527  0.5561  3.3277  0.8935   0.0  0.0000  0.0000  0.0975  0.0000   
103  0.0126  0.0000  0.0000  0.0796   0.0  0.0000  0.0000  0.2538  0.0000   
104  0.9947  0.4070  0.0000  0.1486   0.0  0.0000  0.0000  0.2440  0.0000   

       AAMP  ...  ZSWIM8    ZW10  ZWILCH   ZWINT    ZXDC  ZYG11B     ZYX  \

In [6]:

from sklearn.model_selection import train_test_split

X, y = pd_celldata_t.drop(labels='cancer_type', axis=1),pd_celldata_t['cancer_type']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3,
    stratify=y, random_state=0
)

In [7]:
print(len(X_train))
print(y_train)

73
40    1
89    3
59    1
74    3
61    3
     ..
91    3
79    0
62    4
97    4
49    3
Name: cancer_type, Length: 73, dtype: int64


## Normalization

In [8]:
X_train_norm1 = X_train.div(X_train.sum(axis=1), axis=0)
X_train_norm2 = X_train.div(X_train.sum(axis=0), axis=1)
X_train_norm2=X_train_norm2.fillna(0)

In [9]:
import matplotlib.pyplot as plt
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

scanpy==1.8.2 anndata==0.7.6 umap==0.5.1 numpy==1.19.5 scipy==1.7.1 pandas==1.3.3 scikit-learn==0.24.1 statsmodels==0.13.0rc0 python-igraph==0.9.8 pynndescent==0.5.4


In [10]:
# for adata

import anndata as ad

X_train_a = np.array(X_train_norm1)
adata = ad.AnnData(X_train_a)


adata.uns["name"] = "prot"

y_train_str = []
for i in y_train:
    y_train_str.append(str(i)) # convert to strings so that they can be recognized by scanpy
adata.obs['true_labels'] = y_train_str



## preprocessing 

In [11]:
adata

AnnData object with n_obs × n_vars = 73 × 9733
    obs: 'true_labels'
    uns: 'name'

In [12]:
adata.X

array([[0.00000000e+00, 2.68686737e-04, 6.57511118e-04, ...,
        3.68890527e-04, 2.37525783e-05, 1.14715025e-04],
       [7.44993158e-04, 4.39132418e-04, 0.00000000e+00, ...,
        3.31292540e-05, 1.18933269e-04, 0.00000000e+00],
       [6.35966484e-04, 5.79013315e-04, 3.37664213e-04, ...,
        1.80653151e-04, 4.62237549e-05, 2.60375091e-04],
       ...,
       [3.06582078e-04, 1.59955860e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.54362162e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.42080670e-04, 0.00000000e+00, 0.00000000e+00],
       [1.36017421e-04, 3.56222852e-04, 0.00000000e+00, ...,
        5.06518874e-04, 2.86498871e-05, 1.66336235e-04]], dtype=float32)

In [13]:
adata.obs

,true_labels
0,1
1,3
2,1
3,3
4,3
...,...
68,3
69,0
70,4
71,4


In [14]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        X = self.X_train.iloc[idx].values  # Extract values from X_train DataFrame row
        y = self.y_train.iloc[idx]  # Extract label from y_train DataFrame
        return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.long)  # Convert to PyTorch tensors


In [15]:

def preprocess(adata):
    
    #adata.var_names_make_unique()
        
    
    sc.pp.filter_genes(adata, min_cells=1)
    
    #sc.pp.log1p(adata)


    #sc.pp.pca(adata)

    sc.pp.normalize_total(adata)

    sc.pp.neighbors(adata, use_rep='X')

    '''
    sc.tl.umap(adata)

    lei = sc.tl.leiden(adata, key_added="leiden")

    sc.pl.umap(
        adata,
        color=["leiden"],
        legend_loc="on data",
    )
    '''
    
    
    return adata


In [16]:
print(type(labels[0]))
labels_int = [eval(i) for i in labels]
print(len(labels_int))
labels_y = labels_int[:757]
print(len(labels_y))


<class 'numpy.str_'>
105
105


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np


# Define an Autoencoder (AE) model 
class AE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        latent_representation = self.encoder(x)
        recon_batch = self.decoder(latent_representation)
        return recon_batch, latent_representation

    
# Define a Multi-Layer Perceptron (MLP) model 
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_prob=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.dropout = nn.Dropout(dropout_prob)  # Dropout layer
        self.fc3 = nn.Linear(256, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)
        return x
    
    

In [18]:
gene_rankings = pd.read_csv("rnaSetRanks.csv", sep=',', header=None).to_numpy()
#ordered = order_list(gene_rankings, pd_celldata)

def order_list(gene_rankings, pd_celldata):
    ordered = [None] * len(pd_celldata.columns)
    count = 0
    for col in pd_celldata.columns:
        ordered[count] = 0
        for c in gene_rankings:
            if(c[0] == col):
                ordered[count]=c[1]
        count += 1 
    return ordered


# Define your custom loss function with a list of gene rankings
    
    
class CustomLoss(nn.Module):
    def __init__(self, gene_rankings):
        super(CustomLoss, self).__init__()
        self.gene_rankings = nn.Parameter(torch.Tensor(gene_rankings), requires_grad=True)

    def forward(self, y_true, y_pred):
        # Calculate the weighted loss based on gene rankings
        weighted_loss = torch.sum((1+(1/self.gene_rankings)) * torch.square(y_true - y_pred))
        return weighted_loss

In [19]:
#ordered = order_list(gene_rankings, pd_celldata)
#ordered = ordered[1:]
#ordered = torch.Tensor(ordered)


In [20]:
#ordered.requires_grad_()

In [21]:
#custom_loss = CustomLoss(ordered)
print(len(adata.var))

9733


In [22]:
# Hyperparameters
latent_dim = 64
input_dim = len(adata.var)
output_dim = 10
batch_size = 757

num_epochs = 20

hidden_dim = 256
learning_rate = 0.001


In [23]:

data_name = 'RNA_seq'

# Initialize AE and MLP models with increased complexity and dropout
ae = AE(input_dim, latent_dim)
mlp = MLP(latent_dim, output_dim)

# Combine AE and MLP models
class CombinedModel(nn.Module):
    def __init__(self, ae, mlp):
        super(CombinedModel, self).__init__()
        self.ae = ae
        self.mlp = mlp
    
    def forward(self, x):
        recon_batch, latent_representation = self.ae(x)
        mlp_output = self.mlp(latent_representation)
        return recon_batch, mlp_output

# Create the combined model
combined_model = CombinedModel(ae, mlp)

autoencoder_criterion = nn.MSELoss()
#autoencoder_criterion = nn.BCELoss()

mlp_criterion = nn.CrossEntropyLoss()


optimizer_combined = optim.AdamW(combined_model.parameters(), lr=0.001)

# Initialize the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer_combined, step_size=5, gamma=0.5)


In [24]:
# get data loaders

X_df_train = pd.DataFrame(X_train)
y_df_train = pd.Series(y_train)

dataset_train = MyDataset(X_df_train, y_df_train)
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)



X_df_test = pd.DataFrame(X_test)
y_df_test = pd.Series(y_test)

dataset_test = MyDataset(X_df_test, y_df_test)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


In [25]:
#labels_y=torch.FloatTensor(labels_y)
#labels_y = labels_y.long()
#custom_loss = CustomLoss(ordered)


In [26]:
def composite_loss(autoencoder_loss, mlp_loss, custom_loss_weight):
    return autoencoder_loss + mlp_loss + (custom_loss_weight * custom_loss)


In [27]:
# Define your custom MSE loss function.
class CustomMSELoss(nn.Module):
    def __init__(self, gene_ranking, gene_importance_threshold, learning_rate):
        super(CustomMSELoss, self).__init__()
        self.gene_ranking = nn.Parameter(gene_ranking.clone().detach(), requires_grad=True)
        self.gene_importance_threshold = gene_importance_threshold
        self.learning_rate = learning_rate

    def forward(self, predicted, target):
        mse_loss = torch.mean((predicted - target)**2)

        # Penalize less important genes
        gene_penalty = torch.sum(torch.relu(self.gene_ranking - self.gene_importance_threshold))
        total_loss = mse_loss + gene_penalty

        return total_loss


In [28]:
len(labels)

105

In [29]:
print(type(X_train))
print(type(y_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [30]:
gene_importance_threshold = 0.1
#custom_loss = CustomMSELoss(ordered, gene_importance_threshold, learning_rate)

num_epochs = 50
# Training loop
for epoch in range(num_epochs):
    for batch_idx, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.view(X_train.size(0), -1)

        optimizer_combined.zero_grad()
        recon_batch, mlp_output_train = combined_model(X_train)


        # Calculate AE loss (reconstruction loss)
        #ae_loss = custom_loss(recon_batch, data)
        ae_loss = nn.MSELoss()(recon_batch, X_train)
        

        # Calculate MLP loss 
        mlp_loss = nn.CrossEntropyLoss()(mlp_output_train, y_train)
      
        # Total loss
        total_loss = ae_loss + mlp_loss

        optimizer_combined.zero_grad()
        total_loss.backward(retain_graph=True)
        
        
        optimizer_combined.step()

        #ordered.grad = torch.autograd.grad(ae_loss, custom_loss.gene_ranking)[0]
        
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], AE Loss: {ae_loss.item():.4f}, MLP Loss: {mlp_loss.item():.4f}')

    # Step the learning rate scheduler
    scheduler.step()

    # Evaluate accuracy on validation/test dataset
    combined_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_test, y_test in test_loader:  
            X_test = X_test.view(X_test.size(0), -1)
            _, mlp_output = combined_model(X_test)
            _, predicted = torch.max(mlp_output, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()


            
    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Combined Model Accuracy on Validation/Test dataset: {100 * accuracy:.2f}%')

Epoch [1/50], Batch [0/1], AE Loss: 0.2169, MLP Loss: 2.3182
Epoch [1/50], Combined Model Accuracy on Validation/Test dataset: 15.62%
Epoch [2/50], Batch [0/1], AE Loss: 0.2090, MLP Loss: 2.1662
Epoch [2/50], Combined Model Accuracy on Validation/Test dataset: 31.25%
Epoch [3/50], Batch [0/1], AE Loss: 0.1651, MLP Loss: 1.7080
Epoch [3/50], Combined Model Accuracy on Validation/Test dataset: 31.25%
Epoch [4/50], Batch [0/1], AE Loss: 0.1325, MLP Loss: 1.4161
Epoch [4/50], Combined Model Accuracy on Validation/Test dataset: 21.88%
Epoch [5/50], Batch [0/1], AE Loss: 0.1391, MLP Loss: 1.9354
Epoch [5/50], Combined Model Accuracy on Validation/Test dataset: 28.12%
Epoch [6/50], Batch [0/1], AE Loss: 0.1387, MLP Loss: 2.3649
Epoch [6/50], Combined Model Accuracy on Validation/Test dataset: 34.38%
Epoch [7/50], Batch [0/1], AE Loss: 0.1353, MLP Loss: 1.5370
Epoch [7/50], Combined Model Accuracy on Validation/Test dataset: 31.25%
Epoch [8/50], Batch [0/1], AE Loss: 0.1319, MLP Loss: 1.2987
E

In [31]:

# Convergence criteria
max_iterations = 10  # Adjust as needed
convergence_threshold = 0.001  # Adjust as needed

for iteration in range(max_iterations):
    # Train the model with the current gene rankings
    for epoch in range(num_epochs):
        for batch_idx, (data, labels) in enumerate(train_loader):
            data = data.view(data.size(0), -1)
            # Forward pass, compute loss, and backpropagate
            # Use the custom loss function with the current gene rankings
            optimizer_combined.zero_grad()
            gene_rankings = ordered.clone().detach().requires_grad_(True)
            recon_batch, mlp_output = combined_model(data)
            print(recon_batch)
            print(mlp_output)
            print(labels)
            loss = CustomLoss(ordered)(latent_representation, labels)
            loss.backward()
            optimizer.step()

        # Check for convergence by comparing updated gene rankings
        new_gene_rankings = ordered.detach().numpy()
        if torch.norm(new_gene_rankings - ordered) < convergence_threshold:
            break  # Converged

        # Update gene rankings based on the current model (you can define your update logic)
        ordered = new_gene_rankings

# Final gene rankings after convergence
final_gene_rankings = new_gene_rankings



NameError: name 'ordered' is not defined

In [ ]:
print(ae_loss)
print(mlp_loss)